We first import necessary pakcages needed for our numerical experiment.

In [5]:
import numpy as np
import json
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import psutil
import time
import gurobipy as gp
from gurobipy import GRB

#### Generating Gaussian Distributions 

- generate_gaussians: Randomly generate $K$ tuples $(\mu, \Sigma)$ where $\mu \in \mathbb{R}^d$ is the mean vector and $\Sigma \in \mathbb{R}^{(d \times d)}$ is the covariance matrix. 
INPUT: \
$K \in \mathbb{N}$ - number of Gaussian distributions to generate;\
dim - dimensionality of the Gaussian distributions;\
seed - random seed for reproducibility.\
OUTPUT:\
gaussians - list of tuples, where each tuple contains the mean vector and covariance matrix of a Gaussian distribution.

- generate_gaussian_vectors: Generate random vectors from Gaussian distributions
INPUT:
gaussians - list of tuples, where each tuple contains the mean vector and covariance matrix of a Gaussian distribution;
N - number of random vectors to generate;
seed - random seed for reproducibility.
OUTPUT:
random_vectors - list of random vectors generated from the Gaussian distributions.




In [3]:
def generate_gaussians(K, dim, seed=None):
    np.random.seed(seed)
    gaussians = []
    for _ in range(K):
        # Generate random mean vector
        mean = np.random.uniform(low=-10, high=10, size=(dim,))
        
        # Generate random covariance matrix
        A = np.random.randn(dim, dim)
        covariance_matrix = np.dot(A, A.T)  # Ensures positive semi-definite covariance matrix
        
        gaussians.append((mean, covariance_matrix))
    
    return gaussians

def generate_gaussian_vectors(gaussian, N, seed=None):
    np.random.seed(seed)
    random_vectors = []
    for _ in range(N):
        (mean, covariance_matrix) = gaussian
        # Generate random vector from selected Gaussian distribution
        vector = np.random.multivariate_normal(mean, covariance_matrix)
        random_vectors.append(vector)
    
    return np.array(random_vectors) # sample.shape: (N, dim) 

#### Solving the LP for a low-dimensional optimal coupling.

- solve_OptCoupling_matrix: Solve the stochastic doubly matrix 
$$
\hat{\pi}^\star := \argmin_{\pi \in \Pi(\hat{\mu}, \hat{\nu})}\sum_{i = 1}^m \sum_{j = 1}^n \pi_{ij}\|X_i - Y_j\|^2
$$
corresponding to the optimal coupling between the empirical measures.\
INPUT:\
$x \in \mathbb{R}^{m \times d}$: numpy array\
$y \in \mathbb{R}^{n \times d}$: numpy array\
OUTPUT:\
$\hat{\pi}^\star \in \mathbb{R}^{m \times n}$: numpy array 

In [6]:
def solve_OptCoupling_matrix(x, y):
    m = len(x)
    n = len(y)
    
    # Create a new model
    model = gp.Model("LP_OptCoupling")
    
    # Create decision variables
    pi = {}
    for i in range(m):
        for j in range(n):
            pi[i, j] = model.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name=f"pi_{i}_{j}")
    
    # Update model to integrate new variables
    model.update()
    
    # Set objective function: minimize sum(pi_ij * ||x_i - y_j||^2)
    obj = gp.quicksum(pi[i, j] * np.linalg.norm(x[i] - y[j])**2 for i in range(m) for j in range(n))
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Add constraints: sum(pi_ij) = 1/n for all j
    for j in range(n):
        model.addConstr(gp.quicksum(pi[i, j] for i in range(m)) == 1/n)
    
    # Add constraints: sum(pi_ij) = 1/m for all i
    for i in range(m):
        model.addConstr(gp.quicksum(pi[i, j] for j in range(n)) == 1/m)
    
    # Optimize the model
    model.optimize()
    # breakpoint()
    optimal_solution = np.array([[pi[i, j].x for j in range(n)] for i in range(m)])

    # Output the optimal objective
    optimal_objective = model.objVal
    
    
    return optimal_solution, optimal_objective

#### Solve the QCQP for an optimal $\mathfrak{F}_{\underline{\lambda}, \overline{\lambda}}$-interpolable tuple.

- solve_qcqp: Sole the QCQP to compute the optimal $\mathfrak{F}_{\underline{\lambda}, \overline{\lambda}}$-interpolable tuple.\
INPUT:\
$X \in \mathbb{R}^{(m \times d)}$: numpy array containing the vectors from the first set.\
$Y \in \mathbb{R}^{(n \times d)}$: numpy array containing the vectors from the first set.\
$\hat{\pi}^\star \in \mathbb{R}^{m \times n}$: numpy array containing the optimal coupling.\
$\underline{\lambda}$: lower bound for the lambda parameter.\
$\overline{\lambda}$: upper bound for the lambda parameter.\
OUTPUT:\
optimal_solution_BIg - optimal solution for the BIg variables.
optimal_solution_varphi - optimal solution for the varphi variables.\



In [8]:
def solve_qcqp(BX, BY, hat_pi_star, lambda_lower, lambda_upper):

    m, n = len(BX), len(BY)
    
    # Create a new model
    model = gp.Model("OptTuple_qcqp")
    
    # Set NumericFocus parameter to 3 (Aggressive numerical emphasis)
    model.setParam('NumericFocus', 2)
    
    # Create decision variables
    tilde_BIg = {}
    tilde_varphi = {}
    for i in range(m):
        tilde_BIg[i] = model.addMVar((len(BY[0]),), lb=-GRB.INFINITY)
        tilde_varphi[i] = model.addVar(lb=-GRB.INFINITY)
        # \varphi_i as scalar.
    
    # Update model to integrate new variables
    model.update()
    
    # Set objective function
    obj_expr = gp.QuadExpr()
    for i in range(m):
        for j in range(n):
            if hat_pi_star[i][j] > 1e-5:
                obj_expr += ((BY[j] - tilde_BIg[i] - lambda_lower * BX[i])@(BY[j] - tilde_BIg[i]- lambda_lower * BX[i])) * hat_pi_star[i][j]
            else:
                pass

    model.setObjective(obj_expr, GRB.MINIMIZE)
    # breakpoint()
    
    # Add constraints
    for i in range(m):
        for j in range(m):
            if i != j:
                constraint_expr = gp.QuadExpr()
                inner_product = tilde_BIg[i]@(BX[j] - BX[i])
                norm_squared_tilde_BIg = (tilde_BIg[i] - tilde_BIg[j])@(tilde_BIg[i] - tilde_BIg[j])
                constraint_expr += tilde_varphi[i] -tilde_varphi[j] + inner_product +   (1 / (2*(lambda_upper - lambda_lower))) * norm_squared_tilde_BIg
                model.addConstr(constraint_expr <= 0, "constraint_{}_{}".format(i, j))

    # Optimize the model
    model.optimize()
    
    # Retrieve and print the optimal solution
    if model.status == GRB.OPTIMAL:
        optimal_BIg = np.array([[tilde_BIg[i][j].x + lambda_lower * BX[i][j] for j in range(len(BY[0]))] for i in range(m)])
        optimal_varphi = np.array([tilde_varphi[i].x + lambda_lower**2 / 2 * np.linalg.norm(BX[i])**2 for i in range(m)])
    else:
        print("No optimal solution found")

    return optimal_BIg, optimal_varphi # optimal_BIG.shape = (m x d)


#### Interpolating the optimal shape-constrained least square OT map estimator.

- notation_star: modify the target variables for simplicity of representation.\
INPUT: \
Opt_BIg - optimal solution for the BIg variables\
Opt_varphi - optimal solution for the varphi variables\
BX - m x d matrix (numpy array) containing the vectors from the first set\
lambda_lower - lower bound for the lambda parameter\
lambda_upper - upper bound for the lambda parameter\
OUTPUT:\
tilde_BG - m x d matrix (numpy array) containing the optimal gradient values\
V - m-dimensional vector (numpy array) containing the optimal function values\

- Inter_QP: interpolate the the optimal shape-constrained least square OT map estimator and its corresponding potential function.\
INPUT:\
x - d-dimensional vector (numpy array)\
tilde_BG - m x d matrix (numpy array) containing the optimal gradient values\
tilde_varphi - m-dimensional vector (numpy array) containing the optimal function values\
BX - m x d matrix (numpy array) containing the vectors from the first set\
lambda_lower - lower bound for the lambda parameter\
lambda_upper - upper bound for the lambda parameter\
OUTPUT:\
tilde_BIg_star - interpolated optimal gradient for the SC-LS\
tilde_varphi_star - interpolated optimal function for the SC-LS.


In [10]:
def notation_star(Opt_BIg, Opt_varphi, BX, lambda_lower, lambda_upper):
    m = len(BX)
    tilde_BG = Opt_BIg.copy().T
    V = np.zeros(m).reshape(-1, 1)

    for i in range(m):
        tilde_BG[:, i] = (Opt_BIg[i] - lambda_lower * BX[i])
        # breakpoint()
        v_i = Opt_varphi[i] + 1/(2*(lambda_upper - lambda_lower)) * np.linalg.norm(Opt_BIg[i])**2 + (lambda_lower * lambda_upper / (2 * (lambda_upper - lambda_lower))) * np.linalg.norm(BX[i])**2 - float(lambda_lower / (lambda_upper - lambda_lower)) * (Opt_BIg[i]@ BX[i])
        V[i] = v_i
        # breakpoint()

    return tilde_BG, V

def Interp_QP(x, BX, tilde_BG, V, lambda_lower, lambda_upper):
    m = len(BX)
    
    # Define the model
    model = gp.Model("QP")

    # Define the variables
    BIw = {}
    BIw = model.addMVar((m,), lb = 0, ub = 1, name = "BIw") # BIw as \R^m-vector

    # Define the objective function
    obj_expr = gp.QuadExpr()
    tilde_BG_x_V = tilde_BG.T @ x + V 
    innerprod = tilde_BG_x_V.T @ BIw
    norm_Gw = (tilde_BG @ BIw) @ (tilde_BG @ BIw)
    obj_expr += innerprod - (1 / (2 * (lambda_upper - lambda_lower))) * norm_Gw
    model.setObjective(obj_expr, GRB.MAXIMIZE)

    # Define the constraints
    for i in range(m):
        model.addConstr(BIw.sum() == 1)

    # Optimize the model
    model.optimize()

    # Retrieve and print the optimal solution
    if model.status == GRB.OPTIMAL:
        optimal_weight = BIw.X
        optimal_objective = model.ObjVal  # Optimal objective value
    else:
        print("No optimal solution found")

    tilde_varphi_star = optimal_objective
    tilde_BIg_star = tilde_BG @ optimal_weight
    print("tilde_BIg_star", tilde_BIg_star)
    print("tilde_varphi_star", tilde_varphi_star)

    return tilde_BIg_star, tilde_varphi_star

#### Kernel-based smoothing procedure.

- KS-SCLS_construct: Construct the factors of the $\mathcal{G}_\beta$-smoothed shape-constrained least square estimator and its potential function.\
INPUT:\
BX - samples from the mu;\
BY - samples from the nu;\
lambda_lower - strong convexity parameter;\
lambda_upper - smoothness parameter.\
OUTPUT:\
tilde_BG\
V

- KS_SCLS_compute: Computing the optimal map and the optimal function value given an input $x$.\
INPUT:\
x - random vector;\
BX - samples from the mu;\
lambda_lower - strong convexity parameter;\
lambda_upper - smoothness parameter;\
tilde_BG;\
V;\
Tau - number of iterations;\
beta - smoothing parameter.\
OUTPUT:\
hat_varphi_beta;\
hat_T_beta.

In [11]:
def KS_SCLS_construct(BX, BY, lambda_lower, lambda_upper):
    # Compute the optimal coupling matrix
    hat_pi_star, _ = solve_OptCoupling_matrix(BX, BY)

    # Compute the optimal tuple
    Opt_BIg, Opt_varphi = solve_qcqp(BX, BY, hat_pi_star, lambda_lower, lambda_upper)

    # notations
    tilde_BG, V = notation_star(Opt_BIg, Opt_varphi, BX, lambda_lower, lambda_upper)

    # breakpoint()

    return tilde_BG, V

def KS_SCLS_compute(x, BX, lambda_lower, lambda_upper, tilde_BG, V, Tau = 100, beta = 0.1):
    dim = len(x)
    value_list = []
    map_list = []

    mean_eta = np.zeros(dim)
    covariance_matrix_eta = beta * np.eye(dim)

    count = 0
    for t in range(Tau):
        np.random.seed(t)
        eta = np.random.multivariate_normal(mean_eta, covariance_matrix_eta).reshape(-1, 1)
        tilde_BIg_star, tilde_varphi_star = Interp_QP((x + eta), BX, tilde_BG, V, lambda_lower, lambda_upper)
        value_list.append(tilde_varphi_star)
        map_list.append(tilde_BIg_star)
        count += 1

    tilde_varphi_beta = sum(value_list) / Tau
    tilde_BIg_beta = np.sum(map_list, axis=0) / Tau 

    hat_varphi_beta = tilde_varphi_beta + (lambda_lower/2) * np.linalg.norm(x)**2
    hat_T_beta = tilde_BIg_beta.reshape(-1, 1) + lambda_lower * x

    # breakpoint() 

    return hat_varphi_beta, hat_T_beta

In [12]:
def get_cpu_occupation():
    return psutil.cpu_percent(interval=1)

def read_output(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return np.asarray(data["BX"]), \
            data["lambda_lower"], \
            data["lambda_upper"], \
            np.asarray(data["tilde_BG"]), \
            np.asarray(data["V"])

In [14]:
def objective_compute(X, input_measures, n_samples):
    K = len(input_measures)
    objective = 0
    for k in range(K):
        Y = generate_gaussian_vectors(input_measures[k], n_samples, seed=None)
        _, opt_value = solve_OptCoupling_matrix(X, Y)
        objective += opt_value
    objective /= K
    return objective

#### Present the Proposed Iterative Scheme.

- rho0_sample_generate: Sampling from $\hat{\rho}_0$.
- mu0_sample_generate: Sampling from $\hat{\mu}_0$.
- iterative_sample_generate: generating a single sample from $\hat{\mu}

In [13]:
def rho0_sample_generate(rho0):
    sample = generate_gaussian_vectors(rho0, 1, seed=None)
    return sample.reshape(-1, 1)

def mu0_sample_generate(rho0, R):
    accept = 0
    while accept == 0:
        sample = generate_gaussian_vectors(rho0, 1, seed=None)
        if np.linalg.norm(sample) < R:
            accept = 1
    return sample.reshape(-1, 1) # d x 1

def iterative_sample_generate(K, rho0_sample, iter):
    x = rho0_sample
    # breakpoint()
    for t in range(iter):
        hat_T_beta_sum = np.zeros(len(x)).reshape(-1, 1)
        for k in range(K):
            file_path = "parameters/output_{}_{}.json".format(t, k)
            BX, lambda_lower, lambda_upper, tilde_BG, V = read_output(file_path)
            # breakpoint()
            _, hat_T_beta = KS_SCLS_compute(x, BX, lambda_lower, 
            lambda_upper, tilde_BG, V, Tau = 1000, beta = 0.1)
            # breakpoint()
            hat_T_beta_sum += hat_T_beta
            # breakpoint()
        hat_T_beta_average = hat_T_beta_sum / K
        x = hat_T_beta_average
        # breakpoint()
    return x

def iterative_rejection_sampling(K, rho0, iter, n_samples, R = 1000):
    accepted = []
    while len(accepted) < n_samples:
        rho0_sample = rho0_sample_generate(rho0, R)
        sample = iterative_sample_generate(K, rho0_sample, iter)
        # breakpoint()
        if np.linalg.norm(sample) < R:
            accepted.append(sample)
        print("{}-th sample".format(len(accepted)))
        
    return np.array(accepted)[:, :, 0] #

def iterative_scheme(input_measures, rho0, n_samples):
    K = len(input_measures)
    objective_list = []
    iter = 0
    while iter < 7:
    #while abs(objective - old_objective) > 1e-3:
        # old_objective = objective.copy()
        # breakpoint()
        for k in range(len(input_measures)):
            # breakpoint()
            BX = iterative_rejection_sampling(K, rho0, iter, n_samples) # samples from mu_{iter}
            BY = generate_gaussian_vectors(input_measures[k], n_samples, seed=None) # samples from nu_k
            # breakpoint()

            lambda_lower = 0.1
            lambda_upper = 1000

            tilde_BG, V = KS_SCLS_construct(BX, BY, lambda_lower, lambda_upper)
            output_file = os.path.join("parameters", "output_{}_{}.json".format(iter, k))
            data = {
                "BX": BX.tolist(),
                "BY": BY.tolist(),
                "lambda_lower": lambda_lower,
                "lambda_upper": lambda_upper,
                "tilde_BG": tilde_BG.tolist(),
                "V": V.tolist()
            }
            with open(output_file, 'w') as f:
                json.dump(data, f, indent=4)  # Serialize data to JSON with indentation for readability
            

        output_sample = iterative_rejection_sampling(K, rho0, iter, n_samples)
        objective_list.append(objective_compute(output_sample, input_measures, n_samples))
        iter += 1
        cpu_usage = get_cpu_occupation()
        print("Current CPU usage:", cpu_usage, "%")
        # breakpoint()

    return output_sample, objective_list



In [15]:
K = 5
dim = 2
input_measures = generate_gaussians(K, dim, seed=10)
rho0 = generate_gaussians(1, dim, seed=50)[0] 

output_sample, objective_list = iterative_scheme(input_measures, rho0, n_samples = 30)
print(output_sample)

1-th sample
2-th sample
3-th sample
4-th sample
5-th sample
6-th sample
7-th sample
8-th sample
9-th sample
10-th sample
11-th sample
12-th sample
13-th sample
14-th sample
15-th sample
16-th sample
17-th sample
18-th sample
19-th sample
20-th sample
21-th sample
22-th sample
23-th sample
24-th sample
25-th sample
26-th sample
27-th sample
28-th sample
29-th sample
30-th sample
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-08
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.6.0 21G531)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 60 rows, 900 columns and 1800 nonzeros
Model fingerprint: 0x7b9dc9e4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-02, 3e-02]
Presolve time: 0.02s
Presolved: 60 rows, 900 columns, 1800 nonzeros

Iteration    

KeyboardInterrupt: 